In [1]:
import numpy as np
import math

In [2]:
def top_N(test_u, test_v, test_rate, node_list_u, node_list_v, top_n):
    recommend_dict = {}
    for u in test_u:
        recommend_dict[u] = {}
        for v in test_v:
            if node_list_u.get(u) is None:
                pre = 0
            else:
                U = np.array(node_list_u[u])
                if node_list_v.get(v) is None:
                    pre = 0
                else:
                    V = np.array(node_list_v[v])
                    pre = U.dot(V.T)
            recommend_dict[u][v] = float(pre)

    precision_list = []
    recall_list = []
    ap_list = []
    ndcg_list = []
    rr_list = []

    for u in test_u:
        tmp_r = sorted(recommend_dict[u].items(), key = lambda x: x[1], reverse=True)[0:min(len(recommend_dict[u]),top_n)]
        tmp_t = sorted(test_rate[u].items(), key = lambda x: x[1], reverse=True)[0:min(len(test_rate[u]),len(test_rate[u]))]
      #  tmp_t = sorted(test_rate[u].items(), key = lambda x: x[1], reverse=True)[0:min(len(test_rate[u]), top_n)]
      #  print(tmp_r)
        tmp_r_list = []
        tmp_t_list = []
        for (item, rate) in tmp_r:
            tmp_r_list.append(item)

        for (item, rate) in tmp_t:
            tmp_t_list.append(item)
        pre, rec = precision_and_recall(tmp_r_list,tmp_t_list)
        ap = AP(tmp_r_list,tmp_t_list)
        rr = RR(tmp_r_list,tmp_t_list)
        ndcg = nDCG(tmp_r_list,tmp_t_list)
        precision_list.append(pre)
        recall_list.append(rec)
        ap_list.append(ap)
        rr_list.append(rr)
        ndcg_list.append(ndcg)
    precison = sum(precision_list) / len(precision_list)
    recall = sum(recall_list) / len(recall_list)
    #print(precison, recall)
    f1 = 2 * precison * recall / (precison + recall)
    _map = sum(ap_list) / len(ap_list)
    mrr = sum(rr_list) / len(rr_list)
    mndcg = sum(ndcg_list) / len(ndcg_list)
    return f1,_map,mrr,mndcg

def nDCG(ranked_list, ground_truth):
    dcg = 0
    idcg = IDCG(len(ground_truth))
    for i in range(len(ranked_list)):
        id = ranked_list[i]
        if id not in ground_truth:
            continue
        rank = i+1
        dcg += 1/ math.log(rank+1, 2)
    return dcg / idcg

def IDCG(n):
    idcg = 0
    for i in range(n):
        idcg += 1 / math.log(i+2, 2)
    return idcg

def AP(ranked_list, ground_truth):
    hits, sum_precs = 0, 0.0
    for i in range(len(ranked_list)):
        id = ranked_list[i]
        if id in ground_truth:
            hits += 1
            sum_precs += hits / (i+1.0)
    if hits > 0:
        return sum_precs / len(ground_truth)
    else:
        return 0.0

def RR(ranked_list, ground_list):

    for i in range(len(ranked_list)):
        id = ranked_list[i]
        if id in ground_list:
            return 1 / (i + 1.0)
    return 0

def precision_and_recall(ranked_list,ground_list):
    hits = 0
    for i in range(len(ranked_list)):
        id = ranked_list[i]
        if id in ground_list:
            hits += 1
    pre = hits/(1.0 * len(ranked_list))
    rec = hits/(1.0 * len(ground_list))
    return pre, rec

In [9]:
def read_data(filename_u, filename_i):
    user_vectors = {}
    item_vectors = {}
    with open(filename_u + '.txt', 'r') as fin:
        for line in fin.readlines():
            vec = line.strip().split(' ')
            usr,usr_vec = vec[0], vec[1:]
            usr_vec = np.array(usr_vec, dtype = 'float64')
            user_vectors[usr] = usr_vec
            
    with open(filename_i + '.txt', 'r') as fin:
        for line in fin.readlines():
            vec = line.strip().split(' ')
            item, item_vec = vec[0], vec[1:]
            item_vec = np.array(item_vec, dtype = 'float64')
            item_vectors[item] = item_vec
            
    return user_vectors, item_vectors

In [6]:
def load_test(filename = '../../../BiNE-master/data/rating_test.dat'):
    users,items,rates = set(), set(), {}
    with open(filename, "r", encoding="UTF-8") as fin:
        line = fin.readline()
        while line:
            user, item, rate = line.strip().split()
            if rates.get(user) is None:
                rates[user] = {}
            rates[user][item] = float(rate)
            users.add(user)
            items.add(item)
            line = fin.readline()
        return users, items, rates

In [7]:
test_u, test_v, test_r = load_test()

In [8]:
sorted(test_r['u289'].items(), key = lambda x: x[1], reverse = True)

[('i594', 9.0),
 ('i984', 9.0),
 ('i208', 3.0),
 ('i234', 1.0),
 ('i587', 1.0),
 ('i1427', 1.0),
 ('i1491', 1.0),
 ('i1048', 1.0),
 ('i145', 1.0),
 ('i547', 1.0),
 ('i470', 1.0),
 ('i809', 1.0)]

In [10]:
vec_u, vec_i = read_data('u','v')

In [16]:
f1, _map, mrr, ndcg = top_N(test_u, test_v, test_r, vec_u, vec_i, 10)

In [17]:
print('f1 = ', str(f1*100) + '%')
print('map = ', str(_map*100) + '%')
print('mrr = ', str(mrr*100) + '%')
print('ndcg = ', str(ndcg*100) + '%')

f1 =  0.90050282904182%
map =  1.2717098485969989%
mrr =  2.3390181617818486%
ndcg =  1.8471031881966267%
